In [ ]:
import numpy as np
import pandas as pd
import scipy

In [ ]:
from ecoevocrm.consumer_resource_system import *
from ecoevocrm.landscapes import *
import ecoevocrm.utils as utils
import ecoevocrm.viz as viz

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

#

# Universal parameters

In [ ]:
chromosomeApf = '10000'
chromosomeApc = '01100'
chromosomeBpf = '00001'
chromosomeBpc = '00110'
chromosome_len = 5
plasmid_len = 4

In [ ]:
m = 1e-9
mutation_rates_pf = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])
mutation_rates_pc = np.array([0, 0, 0, 0, 0, 0, m, m, m])

In [ ]:
h = 1
consumption_rates = np.array([h, h, h, h, h, 0, 0, 0, 0])  # for all types
# viz.matrix_plot(consumption_rates, vmin=0, cbar=False, linecolor='lightgray', linewidths=1)

In [ ]:
l = 0 # 1e-7
segregation_rates = np.array([0, 0, 0, 0, 0, l, 0, 0, 0])  # for all types
# viz.matrix_plot(segregation_rates, vmin=0, cbar=False, linecolor='lightgray', linewidths=1)

In [ ]:
beta  = 1e-16
alpha = 1
transfer_rates_donor = np.array([0, 0, 0, 0, 0, beta, 0, 0, 0])  # for all types
transfer_rates_recip = np.array([0, 0, 0, 0, 0, alpha, 0, 0, 0])  # for all types
# viz.matrix_plot(transfer_rates_donor, vmin=0, cbar=False, linecolor='lightgray', linewidths=1)
# viz.matrix_plot(transfer_rates_recip, vmin=0, cbar=False, linecolor='lightgray', linewidths=1)

In [ ]:
linkage = {5: [6, 7, 8]}

In [ ]:
                       #  segregant:  { attributes of segregant }
segregant_overrides = { '011000...': {'traits': np.array([1, 0, 0, 0, 0, 0, 0, 0, 0]), 'mutation_rate': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])},
                        '001100...': {'traits': np.array([0, 0, 0, 0, 1, 0, 0, 0, 0]), 'mutation_rate': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])} }

In [ ]:
                         # transconjugant: { attributes of transconjugant }
transconjugant_overrides = { '100001...': {'traits': {'traits': [0, 1, 2, 3, 4], 'values': np.array([0, 1, 1, 0, 0])}},
                             '000011...': {'traits': {'traits': [0, 1, 2, 3, 4], 'values': np.array([0, 0, 1, 1, 0])}} }

In [ ]:
cost_baseline = 0.1

In [ ]:
N = 3
K = 0

In [ ]:
landscape_hostA = NKLandscape(N=N, K=K, seed=67)
display(landscape_hostA.genotype_fitnesses)

In [ ]:
landscape_hostB = NKLandscape(N=N, K=K, seed=858)
display(landscape_hostB.genotype_fitnesses)

In [ ]:
landscape = merge_landscapes(landscape_hostA.genotype_fitnesses, landscape_hostB.genotype_fitnesses, prefix_A='011001', prefix_B='001101')  # prefixes = traits for pc|A and pc|B

In [ ]:
landscape.update({'100000000': max(landscape_hostA.genotype_fitnesses.values()),   # pf|A
                  '000010000': max(landscape_hostB.genotype_fitnesses.values())})  # pf|B

In [ ]:
landscape

In [ ]:
carrying_capacity = 1e9

# Scenario Phase A1

In [ ]:
                       # h  h  h  h  h   t   m  m  m
traits_init = np.array([[1, 0, 0, 0, 0,  0,  0, 0, 0],   # pf|A
                        [0, 1, 1, 0, 0,  1,  0, 0, 0]])  # pc|A
# viz.matrix_plot(traits_init, vmin=0, cbar=False, linecolor='lightgray', linewidths=1)

In [ ]:
lineageIDs = ['Apf', 'Apc']

In [ ]:
m = 1e-9
mutation_rates = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0],   # pf|A 
                           [0, 0, 0, 0, 0, 0, m, m, m]])  # pc|A
# viz.matrix_plot(mutation_rates, vmin=0, cbar=False, linecolor='lightgray', linewidths=1)

In [ ]:
N_init = np.array([1e8, 1e8])

In [ ]:
R_init = np.array([1, 1, 0, 0, 0, 0, 0, 0, 0])
# viz.matrix_plot(R_init, vmin=0, vmax=1, cbar=False, linecolor='lightgray', linewidths=1)

In [ ]:
influx_rates = np.zeros_like(R_init)
influx_rates[R_init > 0] = 1
# viz.matrix_plot(influx_rates, vmin=0, cbar=False, linecolor='lightgray', linewidths=1)

In [ ]:
community_envA1 = Community(traits=traits_init, cost_landscape=landscape, cost_baseline=cost_baseline,
                      consumption_rate=consumption_rates, influx_rate=influx_rates, carrying_capacity=carrying_capacity, 
                      mutation_rate=mutation_rates, segregation_rate=segregation_rates, transfer_rate_donor=transfer_rates_donor, transfer_rate_recip=transfer_rates_recip,
                      segregant_overrides=segregant_overrides, transconjugant_overrides=transconjugant_overrides,
                      segregation_linkage=linkage, transfer_linkage=linkage,
                      lineageIDs=lineageIDs, lineageID_traits=[6, 7, 8],
                      N_init=N_init, R_init=R_init, seed=2)

In [ ]:
T_total = 1e3

In [ ]:
community_envA1.run(T=T_total)

In [ ]:
cmapA = viz.lightdark_cmap('crimson', cmin=0.1, cmax=0.95, reverse=True)
cmapB = viz.lightdark_cmap('royalblue', cmin=0.1, cmax=0.95, reverse=True)
xconj_cmaps = [viz.lightdark_cmap(sns.color_palette('gnuplot2', as_cmap=True)(np.arange(0.45, 1.0, 0.1)[n-1]), cmin=0.1, cmax=0.95, reverse=True) for n in range(1, 7)]
# display(cmapA)
# display(cmapB)
# for cmap in xconj_cmaps: display(cmap)

In [ ]:
type_colors, type_hatches = viz.type_styles_by_phylogeny(community_envA1.type_set, color_tags={'A': cmapA, 'B': cmapB, 't': {n+1: cmapT for n, cmapT in enumerate(xconj_cmaps)}}, hatch_tags={'s': '///', 'pf': '///'}, 
                                                         vmin=min(landscape.values()), vmax=max(landscape.values()) )

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 6))
viz.abundance_plot(community_envA1, ax=ax, type_colors=type_colors, type_hatches=type_hatches, relative_abundance=False, stacked=True, baseline='sym', log_x_axis=True, log_y_axis=False)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
viz.resource_plot(community_envA1, ax=ax, stacked=True, relative=False, log_x_axis=True, log_y_axis=False)

# Scenario Phase B2

In [ ]:
traits_init = []
lineageIDs  = []
mutation_rates = []
for tidx in range(community_envA1.num_types):
    traitStr = community_envA1.type_set.trait_keys[tidx]
    plasmidStr = traitStr[-plasmid_len:]
    if(traitStr[:chromosome_len] == chromosomeApc):
        traits_init.append([int(c) for c in chromosomeBpc+plasmidStr])
        lineageIDs.append(f'{community_envA1.type_set.lineageIDs[tidx]}>{"Bpc"}[{plasmidStr[1:]}]')
        mutation_rates.append(mutation_rates_pc)
    elif(traitStr[:chromosome_len] == chromosomeApf):
        traits_init.append([int(c) for c in chromosomeBpf+plasmidStr])
        lineageIDs.append(f'{community_envA1.type_set.lineageIDs[tidx]}>{"Bpf"}[{plasmidStr[1:]}]')
        mutation_rates.append(mutation_rates_pf)        
traits_init = np.array(traits_init)
mutation_rates = np.array(mutation_rates)
print(traits_init)
print(mutation_rates)

In [ ]:
lineageIDs = None

In [ ]:
N_init = community_envA1.N

In [ ]:
R_init = np.array([0, 0, 0, 1, 1, 0, 0, 0, 0])

In [ ]:
influx_rates = np.zeros_like(R_init)
influx_rates[R_init > 0] = 1

In [ ]:
community_envB2 = Community(traits=traits_init, cost_landscape=landscape, cost_baseline=cost_baseline,
                      consumption_rate=consumption_rates, influx_rate=influx_rates, carrying_capacity=carrying_capacity, 
                      mutation_rate=mutation_rates, segregation_rate=segregation_rates, transfer_rate_donor=transfer_rates_donor, transfer_rate_recip=transfer_rates_recip,
                      segregant_overrides=segregant_overrides, transconjugant_overrides=transconjugant_overrides,
                      segregation_linkage=linkage, transfer_linkage=linkage,
                      lineageIDs=lineageIDs, lineageID_traits=[6, 7, 8],
                      N_init=N_init, R_init=R_init, seed=2)

In [ ]:
T_total = 1e4

In [ ]:
community_envB2.run(T=T_total)

In [ ]:
cmapA = viz.lightdark_cmap('crimson', cmin=0.1, cmax=0.95, reverse=True)
cmapB = viz.lightdark_cmap('royalblue', cmin=0.1, cmax=0.95, reverse=True)
xconj_cmaps = [viz.lightdark_cmap(sns.color_palette('gnuplot2', as_cmap=True)(np.arange(0.45, 1.0, 0.1)[n-1]), cmin=0.1, cmax=0.95, reverse=True) for n in range(1, 7)]

In [ ]:
type_colors, type_hatches = viz.type_styles_by_phylogeny(community_envB2.type_set, color_tags={'A': cmapA, 'B': cmapB, 't': {n+1: cmapT for n, cmapT in enumerate(xconj_cmaps)}}, hatch_tags={'s': '///', 'pf': '///'}, vmin=min(landscape.values()), vmax=max(landscape.values()), palette_depth=0 )

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 6))
viz.abundance_plot(community_envB2, ax=ax, type_colors=type_colors, type_hatches=type_hatches, relative_abundance=False, stacked=True, baseline='sym', log_x_axis=True, log_y_axis=False)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
viz.resource_plot(community_envB2, ax=ax, stacked=True, relative=False, log_x_axis=True, log_y_axis=False)

# Scenario Phase A3

In [ ]:
traits_init = []
lineageIDs  = []
mutation_rates = []
for tidx in range(community_envB2.num_types):
    traitStr = community_envB2.type_set.trait_keys[tidx]
    plasmidStr = traitStr[-plasmid_len:]
    if(traitStr[:chromosome_len] == chromosomeBpc):
        traits_init.append([int(c) for c in chromosomeApc+plasmidStr])
        lineageIDs.append(f'{community_envB2.type_set.lineageIDs[tidx]}>{"Apc"}[{plasmidStr[1:]}]')
        mutation_rates.append(mutation_rates_pc)
    elif(traitStr[:chromosome_len] == chromosomeBpf):
        traits_init.append([int(c) for c in chromosomeApf+plasmidStr])
        lineageIDs.append(f'{community_envB2.type_set.lineageIDs[tidx]}>{"Apf"}[{plasmidStr[1:]}]')
        mutation_rates.append(mutation_rates_pf)   
traits_init = np.array(traits_init)
mutation_rates = np.array(mutation_rates)
print(traits_init)
print(mutation_rates)

In [ ]:
lineageIDs = None

In [ ]:
N_init = community_envB2.N

In [ ]:
R_init = np.array([1, 1, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
influx_rates = np.zeros_like(R_init)
influx_rates[R_init > 0] = 1

In [ ]:
community_envA3 = Community(traits=traits_init, cost_landscape=landscape, cost_baseline=cost_baseline,
                      consumption_rate=consumption_rates, influx_rate=influx_rates, carrying_capacity=carrying_capacity, 
                      mutation_rate=mutation_rates, segregation_rate=segregation_rates, transfer_rate_donor=transfer_rates_donor, transfer_rate_recip=transfer_rates_recip,
                      segregant_overrides=segregant_overrides, transconjugant_overrides=transconjugant_overrides,
                      segregation_linkage=linkage, transfer_linkage=linkage,
                      lineageIDs=lineageIDs, lineageID_traits=[6, 7, 8],
                      N_init=N_init, R_init=R_init, seed=2)

In [ ]:
T_total = 1e4

In [ ]:
community_envA3.run(T=T_total)

In [ ]:
cmapA = viz.lightdark_cmap('crimson', cmin=0.1, cmax=0.95, reverse=True)
cmapB = viz.lightdark_cmap('royalblue', cmin=0.1, cmax=0.95, reverse=True)
xconj_cmaps = [viz.lightdark_cmap(sns.color_palette('gnuplot2', as_cmap=True)(np.arange(0.45, 1.0, 0.1)[n-1]), cmin=0.1, cmax=0.95, reverse=True) for n in range(1, 7)]

In [ ]:
type_colors, type_hatches = viz.type_styles_by_phylogeny(community_envA3.type_set, color_tags={'A': cmapA, 'B': cmapB, 't': {n+1: cmapT for n, cmapT in enumerate(xconj_cmaps)}}, hatch_tags={'s': '///', 'pf': '///'}, vmin=min(landscape.values()), vmax=max(landscape.values()), palette_depth=0 )

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 6))
viz.abundance_plot(community_envA3, ax=ax, type_colors=type_colors, type_hatches=type_hatches, relative_abundance=False, stacked=True, baseline='sym', log_x_axis=True, log_y_axis=False)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
viz.resource_plot(community_envA3, ax=ax, stacked=True, relative=False, log_x_axis=True, log_y_axis=False)

In [ ]:
community_envA3.extant_type_set.traits

In [ ]:
community_envA3.N[community_envA3.get_extant_type_indices()]